In [1]:
MAX_COUNT = 25

In [2]:
import os
os.chdir("Data")
!ls

Challenge           leaderboard.h5      odor_pair_full.json test.h5
dataset.h5          molecules.csv       pair_huge.json      train.h5


In [3]:
import json
with open("pair_huge.json") as f:
    dataset = json.load(f)

len(dataset),dataset[0]

(266758,
 {'mol1': 'CC(=CCC1C(O1)(C)C)C=C',
  'mol2': 'CCCC(CC)O',
  'blend_notes': ['herbal']})

In [4]:
import tqdm

all_notes = set()
for d in tqdm.tqdm(dataset):
    all_notes.update(d["blend_notes"])
all_notes = list(all_notes)

all_notes

100%|████████████████████████████| 266758/266758 [00:00<00:00, 4325767.72it/s]


['bitter',
 'milky',
 'winey',
 'chocolate',
 'dry',
 'vanilla',
 'coumarinic',
 'fruity',
 'terpenic',
 'tarragon',
 'musk',
 'powdery',
 'clean',
 'tropical',
 'whiskey',
 'alliaceous',
 'medicinal',
 'oily',
 'spicy',
 'seafood',
 'honey',
 'celery',
 'rooty',
 'burnt',
 'astringent',
 'cherry',
 'tomato',
 'berry',
 'phenolic',
 'dairy',
 'acidic',
 'mushroom',
 'vegetable',
 'caramellic',
 'melon',
 'meaty',
 'savory',
 'fishy',
 'greasy',
 'leafy',
 'creamy',
 'ketonic',
 'malty',
 'pungent',
 'sour',
 'green',
 'kokumi',
 'tobacco',
 'dusty',
 'cucumber',
 'odorless',
 'peach',
 'potato',
 'naphthyl',
 'orris',
 'nutty',
 'leathery',
 'fungal',
 'estery',
 'acrylate',
 'eggy',
 'cooling',
 'fusel',
 'musty',
 'chemical',
 'lactonic',
 'balsamic',
 'buttery',
 'coffee',
 'hay',
 'metallic',
 'minty',
 'wasabi',
 'apple',
 'acetic',
 'mustard',
 'sweet',
 'citrus',
 'marine',
 'camphoreous',
 'cabbage',
 'corn',
 'aromatic',
 'woody',
 'alcoholic',
 'licorice',
 'corn chip',
 'fre

In [5]:
import torch
# Create a dictionary mapping each label to a unique index
label_to_index = {label: idx for idx, label in enumerate(all_notes)}

def multi_hot(notes):
    # Initialize a zero tensor of the appropriate size
    multi_hot_vector = torch.zeros(len(all_notes))

    # Set the corresponding positions in the tensor to 1 for each label of the item
    for label in notes:
        index = label_to_index[label]
        multi_hot_vector[index] = 1
    return multi_hot_vector

all_multihots = dict()
for d in tqdm.tqdm(dataset):
    all_multihots[(d["mol1"],d["mol2"])] = multi_hot(d["blend_notes"])
next(iter(all_multihots.items()))

100%|█████████████████████████████| 266758/266758 [00:01<00:00, 219482.56it/s]


(('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]))

In [6]:
import collections

notes_to_pairs = collections.defaultdict(list)
for d in tqdm.tqdm(dataset):
    notes_to_pairs[frozenset(d["blend_notes"])].append((d["mol1"],d["mol2"]))

len(notes_to_pairs), len(next(iter(notes_to_pairs.values())))

100%|████████████████████████████| 266758/266758 [00:00<00:00, 2683584.37it/s]


(6850, 4384)

In [7]:
import random

positives = dict()
for n, pairs in tqdm.tqdm(notes_to_pairs.items()):
    for p1 in pairs:
        vals = list(random.sample(pairs,MAX_COUNT)) if len(pairs) > MAX_COUNT else pairs
        positives[p1] = [p2 for p2 in vals if p1 != p2]
len(positives), next(iter(positives.items()))

100%|███████████████████████████████████| 6850/6850 [00:01<00:00, 3641.14it/s]


(266758,
 (('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
  [('CCCC1CC(CCO1)(C)OC(=O)C', 'COC(=O)CCC1=CC=CC=C1'),
   ('CC(C)C1C(COCO1)(C)C',
    'C[C@@H]1CC[C@@H]2[C@]13CC[C@@]([C@H](C3)C2(C)C)(C)O'),
   ('CCCCCCCCC(=O)OC/C=C(\\C)/CCC=C(C)C', 'CCOC(=O)C1C(C1(C)C)C=C(C)C'),
   ('CC(C)CC(CC(C)C)OC(=O)C', 'CC1C=C(CCC1C(=O)C(C)(C)C)C'),
   ('CC1=CC(=CC(=C1)OC)O', 'CCC1=CC=C(C=C1)CC(C)(C)C#N'),
   ('CCCC/C=C/CO', 'CCCCCCOC(=O)\\C=C\\C(=O)OCCCCCC'),
   ('CCC(C)(CCCC(C)C=O)OC', 'CC[C@@H](C)C(=O)OCC'),
   ('CC(C)CCC1(OCC=CCO1)C', 'CCCCC(CC)O'),
   ('C1=CC=C(C(=C1)C=O)O', 'CCCCC(CO)C(=O)C'),
   ('CC(C)C(=O)OC(C)CC(=C)C', 'C[C@]1(CC[C@@H](O1)C(C)(C)O)C=C'),
   ('CC(=O)OC1CC2CC1C3C2CCC3', 'CCCCCC(=O)OC(C)(CCC=C(C)C)C=C'),
   ('CC(C)(CC1=CC=CC=C1)OC=O', 'CC(C)CC(CC1=CC=CC=C1)O'),
   ('CC(C)(C)C1CCCCC1O', 'CC1(C2CCC(C2)(C1=O)C)C'),
   ('CC(=C)CCOC(=O)C(C)(C)C', 'CCCCC1OC(CC(O1)(C)C)C'),
   ('CC(C)CC(=O)OCC(=Cc1ccccc1)CCCCC', 'CC1=CCCC(C1C(=O)OC)(C)C'),
   ('CC1CC2C3CCC(C3)C2CC1=O', 'CCCCOC(=O)CCC(=O)C'),


In [8]:
missing = len([p for p, ps in positives.items() if len(ps) < MAX_COUNT])
missing, missing/len(dataset)

(32778, 0.12287541517030418)

In [9]:
mol_sets = collections.defaultdict(list)
for i,d in enumerate(dataset):
    mol_sets[d["mol1"]].append(i)
    mol_sets[d["mol2"]].append(i)

In [10]:
pairs = []
pair_to_notes = dict()
for d in dataset:
    pair = (d["mol1"],d["mol2"])
    pairs.append(pair)
    pair_to_notes[pair] = set(d["blend_notes"])

In [11]:
notes_sets = dict()
for i,d in enumerate(dataset):
    notes_sets[i] = set(d["blend_notes"])

In [12]:
# Hard negatives are anchor/negatives that share a molecule
# but do not have any notes in common.
negatives = collections.defaultdict(list)
for (mol,idcs) in tqdm.tqdm(mol_sets.items()):
    for i in idcs:
        p1 = notes_sets[i]
        for j in idcs:
            p2 = notes_sets[j]
            if bool(p1 & p2):
                continue
            negatives[pairs[i]].append(pairs[j])
len(negatives)

100%|████████████████████████████████████| 3428/3428 [00:26<00:00, 130.44it/s]


266242

In [13]:
missing = len([p for p, ns in negatives.items() if len(ns) < MAX_COUNT])
missing, missing/len(dataset)

(15554, 0.058307529671087654)

In [14]:
# # Supplement with random negatives if we have less than the required
# # number of hard negatives.
# for p, ns in tqdm.tqdm(negatives.items()):
#     notes = pair_to_notes[pair]
#     while len(ns) < MAX_COUNT:
#         other = random.choice(dataset)
#         other_pair = (other["mol1"],other["mol2"])
#         other_notes = pair_to_notes[other_pair]
#         if bool(notes & other_notes):
#             continue
#         ns.append(other_pair)
#     negatives[p] = list(random.sample(pairs,MAX_COUNT))

# missing = len([p for p, ns in negatives.items() if len(ns) != MAX_COUNT])
# len(negatives), missing, missing/len(dataset)      

In [15]:
next(iter(negatives.items()))

(('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
 [('CC(=CCC1C(O1)(C)C)C=C', 'CCOC(C)(CCC=C(C)C)C=C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC/C=C\\CCOC(=O)C1=CC=CC=C1O'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC(C)(CCC1CCCCC1)O'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC(=CCC[C@@](C)(C=C)O)C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCOC(C)OC(C)(CCC=C(C)C)C=C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC/C(=C/CCC(C)(C=C)OC(=O)C)/C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC/C(=C/CCC(C)(C=C)O)/C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCC(C)(CCC=C(C)C)O'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC1CCOC(C1)CC(C)C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC1=CCC(=C(C)CCC=C(C)C)CC1'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCCCCCCCOC(=O)CC1=CC=CC=C1'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CC1(CCC(C23C14C2CC(C4)C3(C)C)COC=O)C'),
  ('CC(=CCC1C(O1)(C)C)C=C', 'CCC(C)(CCCC(C)C)O'),
  ('CC(=O)OCC1CCC(C23C14C2CC(C3)C4(C)C)(C)C', 'CCCC(CC)O'),
  ('CCC(C)C=O', 'CCCC(CC)O'),
  ('CC1(CCC(C23C14C2CC(C4)C3(C)C)CO)C', 'CCCC(CC)O'),
  ('CCCC(CC)O', 'CCOC(=O)C(C)C'),
  ('CCCC(CC)O', 'C[C@]12CCCC(C1CC[C@@]([C@@H

In [16]:
all_smiles = set()
for d in dataset:
    all_smiles.add(d["mol1"])
    all_smiles.add(d["mol2"])
len(all_smiles),next(iter(all_smiles))

(3428, 'C[C@@H]1CC[C@@H](C(=O)C1)C(C)(C)S')

In [17]:
from ogb.utils import smiles2graph
from torch_geometric.data import InMemoryDataset, download_url, Data
from torch_geometric.loader import DataLoader

def to_torch(graph):
    tensor_keys = ["edge_index", 'edge_feat', 'node_feat']
    for key in tensor_keys:
        graph[key] = torch.from_numpy(graph[key])
    return Data(x=graph["node_feat"].float(),edge_attr=graph["edge_feat"],edge_index=graph["edge_index"])

errored = 0
graph_data = dict()
for smiles in tqdm.tqdm(all_smiles):
    try:
        graph_data[smiles] = to_torch(smiles2graph(smiles))
    except AttributeError as e:
        errored += 1
errored, len(graph_data), next(iter(graph_data.items()))

 51%|██████████████████                 | 1765/3428 [00:01<00:01, 1022.25it/s][11:20:24] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[11:20:24] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'
 97%|█████████████████████████████████▉ | 3320/3428 [00:03<00:00, 1028.19it/s][11:20:25] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[11:20:25] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'
100%|███████████████████████████████████| 3428/3428 [00:03<00:00, 1029.73it/s]


(3,
 3425,
 ('C[C@@H]1CC[C@@H](C(=O)C1)C(C)(C)S',
  Data(x=[12, 9], edge_index=[2, 24], edge_attr=[24, 3])))

In [18]:
import numpy as np
import data

pair_to_data = dict()
for d in tqdm.tqdm(dataset):
    if not d["mol1"] in graph_data or not d["mol2"] in graph_data:
        continue
    pair = (d["mol1"],d["mol2"])
    g1 = graph_data[d["mol1"]]
    g2 = graph_data[d["mol2"]]
    pair_to_data[pair] = data.combine_graphs([g1,g2])
len(pair_to_data), next(iter(pair_to_data.items()))

100%|███████████████████████████████| 266758/266758 [00:55<00:00, 4777.63it/s]


(266450,
 (('CC(=CCC1C(O1)(C)C)C=C', 'CCCC(CC)O'),
  BlendData(x=[18, 9], edge_index=[2, 34], edge_attr=[34, 3], mol_batch=[18], blend_batch=[2])))

In [19]:
valid_pairs = set(pair_to_data.keys()).intersection(set(positives.keys())).intersection(set(negatives.keys())).intersection(set(all_multihots.keys()))
len(valid_pairs), len(valid_pairs)/len(dataset)

(265934, 0.9969110579626478)

In [23]:
import json
import h5py
import base64

def encode_smiles(smiles):
    return base64.urlsafe_b64encode(smiles.encode()).decode()

with h5py.File('dataset_contrast.h5', 'w') as f:
    for (pair, data) in tqdm.tqdm(pair_to_data.items()):
        group = f.create_group(encode_smiles(json.dumps(pair)))
        group.create_dataset("pair",data=pairs[i])
        group.create_dataset("positives",data=positives[pair])
        group.create_dataset("negatives",data=negatives[pair])
        graph_group = group.create_group("graph")
        for k,v in data.items():
            graph_group.create_dataset(k,data=v.numpy())

100%|████████████████████████████████| 266450/266450 [08:37<00:00, 515.24it/s]


In [24]:
import json
import h5py
import base64

def encode_smiles(smiles):
    return base64.urlsafe_b64encode(smiles.encode()).decode()

with h5py.File('dataset_labels.h5', 'w') as f:
    for (pair, data) in tqdm.tqdm(pair_to_data.items()):
        group = f.create_group(encode_smiles(json.dumps(pair)))
        group.create_dataset("pair",data=pairs[i])
        group.create_dataset("notes",data=all_multihots[pair].numpy())
        graph_group = group.create_group("graph")
        for k,v in data.items():
            graph_group.create_dataset(k,data=v.numpy())

100%|███████████████████████████████| 266450/266450 [02:55<00:00, 1521.87it/s]
